In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from sklearn.metrics import f1_score
import keras.backend as K

In [65]:
# Set the image and model parameters
image_width, image_height = 128, 128
num_classes = 2
batch_size = 32
epochs = 10

# Define the image directories
image_dir = 'C:/Users/User/Apviza/Project 4/images'
training_dir = os.path.join(image_dir, 'training')
testing_dir = os.path.join(image_dir, 'testing')


In [38]:
# Define the image directories
image_dir = 'C:/Users/User/Apviza/Project 4/images'
training_dir = os.path.join(image_dir, 'training')
testing_dir = os.path.join(image_dir, 'testing')


# Load and preprocess the images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(image_width, image_height))
    img = img_to_array(img) / 255.0  # Normalize pixel values between 0 and 1
    return img

def load_images_from_folder(folder):
    images = []
    labels = []
    flip_folder = os.path.join(folder, 'flip')
    not_flip_folder = os.path.join(folder, 'notflip')

    # Load images from the 'flip' folder
    for filename in os.listdir(flip_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = preprocess_image(os.path.join(flip_folder, filename))
            images.append(img)
            labels.append(1)  # Flipped page

    # Load images from the 'notflip' folder
    for filename in os.listdir(not_flip_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img = preprocess_image(os.path.join(not_flip_folder, filename))
            images.append(img)
            labels.append(0)  # Not flipped page

    return np.array(images), np.array(labels)

In [39]:
# Load and split the dataset
X_train, y_train = load_images_from_folder(training_dir)
X_test, y_test = load_images_from_folder(testing_dir)

# Perform one-hot encoding on the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Split training set into training and validation

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Fix image size if necessary
def fix_image_size(image):
    return tf.image.resize(image, [image_width, image_height])

X_train = fix_image_size(X_train)
X_val = fix_image_size(X_val)
X_test = fix_image_size(X_test)

In [24]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [10]:
# Design the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy",f1_metric])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/10
60/60 [==============================] - 16s 246ms/step - loss: 0.6875 - accuracy: 0.5609 - f1_metric: 0.5632 - val_loss: 0.5985 - val_accuracy: 0.6764 - val_f1_metric: 0.6627
Epoch 2/10
60/60 [==============================] - 14s 240ms/step - loss: 0.4637 - accuracy: 0.7716 - f1_metric: 0.7738 - val_loss: 0.4361 - val_accuracy: 0.7975 - val_f1_metric: 0.7931
Epoch 3/10
60/60 [==============================] - 15s 248ms/step - loss: 0.2376 - accuracy: 0.8986 - f1_metric: 0.8974 - val_loss: 0.1635 - val_accuracy: 0.9499 - val_f1_metric: 0.9482
Epoch 4/10
60/60 [==============================] - 15s 249ms/step - loss: 0.1285 - accuracy: 0.9524 - f1_metric: 0.9532 - val_loss: 0.2062 - val_accuracy: 0.9269 - val_f1_metric: 0.9236
Epoch 5/10
60/60 [==============================] - 15s 250ms/step - loss: 0.1197 - accuracy: 0.9556 - f1_metric: 0.9525 - val_loss: 0.3075 - val_accuracy: 0.8685 - val_f1_metric: 0.8718
Epoch 6/10
60/60 [==============================] - 15s 242ms/ste

In [6]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

f1 = f1_score(y_test, y_pred, average='micro')
print("F1 Score:", f1)

19/19 [==============================] - 1s 52ms/step
F1 Score: 0.964765100671141


In [30]:
# Define the image directories
image_dir = 'C:/Users/User/Apviza/Project 4/images'
training_dir = os.path.join(image_dir, 'training')
testing_dir = os.path.join(image_dir, 'testing')

# Parameters
image_width = 64
image_height = 64
batch_size = 1
epochs = 10

# Load and preprocess the images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(image_width, image_height))
    img = img_to_array(img) / 255.0  # Normalize pixel values between 0 and 1
    return img

def load_sequences_from_folder(folder):
    sequences = []
    labels = []

    flip_folder = os.path.join(folder, 'flip')
    not_flip_folder = os.path.join(folder, 'notflip')

    # Load sequences from the 'flip' folder
    for filename in sorted(os.listdir(flip_folder)):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(flip_folder, filename)
            sequence_id = filename.split('_')[0]
            frame_number = filename.split('_')[1].split('.')[0]
            sequence = sequences[-1][1] if sequences else []
            sequence.append((frame_number, preprocess_image(image_path)))
            if not sequences or sequence_id != sequences[-1][0]:
                sequences.append((sequence_id, sequence))

    # Load sequences from the 'notflip' folder
    for filename in sorted(os.listdir(not_flip_folder)):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(not_flip_folder, filename)
            sequence_id = filename.split('_')[0]
            frame_number = filename.split('_')[1].split('.')[0]
            sequence = sequences[-1][1] if sequences else []
            sequence.append((frame_number, preprocess_image(image_path)))
            if not sequences or sequence_id != sequences[-1][0]:
                sequences.append((sequence_id, sequence))

    # Sort the frames within each sequence by frame number
    for _, sequence in sequences:
        sequence.sort(key=lambda x: int(x[0]))

    # Create labels based on the presence of 'flip' or 'notflip' folder
    for sequence in sequences:
        if os.path.exists(os.path.join(flip_folder, f"{sequence[1][0][0]}.jpg")):
            labels.append(1)  # Flipped page
        else:
            labels.append(0)  # Not flipped page

    return sequences, np.array(labels)

# Define the data generator for training
def data_generator(sequences, labels, batch_size):
    num_batches = len(sequences) // batch_size
    while True:
        for i in range(num_batches):
            batch_sequences = sequences[i*batch_size:(i+1)*batch_size]
            batch_labels = labels[i*batch_size:(i+1)*batch_size]
            train_sequences_processed = np.array([np.array([frame[1] for frame in sequence]) for _, sequence in batch_sequences])
            train_labels_processed = batch_labels
            yield train_sequences_processed, train_labels_processed

In [26]:
# Load training sequences and labels
train_sequences, train_labels = load_sequences_from_folder(training_dir)

# Load testing sequences and labels
test_sequences, test_labels = load_sequences_from_folder(testing_dir)

In [27]:
# Design the RNN model architecture
sq_model = tf.keras.Sequential()
sq_model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu'), input_shape=(None, image_width, image_height, 3)))
sq_model.add(TimeDistributed(MaxPooling2D((2, 2))))
sq_model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
sq_model.add(TimeDistributed(MaxPooling2D((2, 2))))
sq_model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
sq_model.add(TimeDistributed(MaxPooling2D((2, 2))))
sq_model.add(TimeDistributed(Flatten()))
sq_model.add(LSTM(32, return_sequences=True))
sq_model.add(LSTM(32))
sq_model.add(Dense(1, activation='sigmoid'))

# Compile the model
sq_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
# Create the data generator
train_data_generator = data_generator(train_sequences, train_labels, batch_size)

# Train the model using the data generator
steps_per_epoch = len(train_sequences) // batch_size
#sq_model.fit(train_data_generator, steps_per_epoch=steps_per_epoch, epochs=epochs)
sq_model.fit(train_data_generator, steps_per_epoch=steps_per_epoch, epochs=epochs)

# Preprocess testing sequences
test_sequences_processed = np.array([np.array([frame[1] for frame in sequence]) for _, sequence in test_sequences])
test_labels_processed = test_labels

Epoch 1/10
117/117 [==============================] - 350s 3s/step - loss: 0.0018 - accuracy: 1.0000
Epoch 2/10
117/117 [==============================] - 349s 3s/step - loss: 9.0966e-04 - accuracy: 1.0000
Epoch 3/10
117/117 [==============================] - 348s 3s/step - loss: 5.7137e-04 - accuracy: 1.0000
Epoch 4/10
117/117 [==============================] - 338s 3s/step - loss: 4.0497e-04 - accuracy: 1.0000
Epoch 5/10
117/117 [==============================] - 351s 3s/step - loss: 3.0688e-04 - accuracy: 1.0000
Epoch 6/10
117/117 [==============================] - 350s 3s/step - loss: 2.4215e-04 - accuracy: 1.0000
Epoch 7/10
117/117 [==============================] - 349s 3s/step - loss: 1.9661e-04 - accuracy: 1.0000
Epoch 8/10
117/117 [==============================] - 349s 3s/step - loss: 1.6307e-04 - accuracy: 1.0000
Epoch 9/10
117/117 [==============================] - 348s 3s/step - loss: 1.3752e-04 - accuracy: 1.0000
Epoch 10/10
117/117 [==============================] - 350s

In [33]:
test_loss, test_accuracy = sq_model.evaluate(test_sequences_processed, test_labels, verbose=0)
predictions = sq_model.predict(test_sequences_processed)
f1score = f1_score(test_labels, predictions.round(), zero_division=1)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)
print('F1 Score:', f1score)

4/4 [==============================] - 26s 6s/step
Test Loss: 0.00010893327998928726
Test Accuracy: 1.0
F1 Score: 1.0
